<a href="https://colab.research.google.com/github/moathzyout/Project/blob/main/DL-approaches-text%20and%20binary%20encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

**Two main tasks to classify Mal-prem dataset are developed in this work to classify malware-based binary encoding and text encoding for permissions using LSTM and Convd1d. In addition, different algorithms are presented on how select the best batch size in all tasks by the Keraseclassier python package.**

In [ ]:

from sklearn import metrics
import re
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Bidirectional,MaxPool3D,MaxPool2D,SpatialDropout1D,UpSampling2D,Embedding,Reshape,MaxPooling2D,Convolution2D,Convolution1D,Input,LSTM,Dropout,Flatten,BatchNormalization,MaxPooling1D,GlobalMaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam,RMSprop
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,BatchNormalization,Masking,UpSampling2D,Reshape,MaxPooling2D,Convolution2D,Convolution1D,Input,LSTM,Dropout,Flatten,BatchNormalization,MaxPooling1D,GlobalMaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#from _typeshed import OpenTextMode

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()



In [ ]:
from keras.backend import batch_get_value
def plot_history_opt(history,opt,batch):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy for '+ opt+' with '+batch)
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss for '+ opt+' with '+batch)
    plt.legend()

In [ ]:
import sklearn
import seaborn as sns 

import matplotlib.pyplot as plt 
def plot_history_loss(history):
   
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(loss) + 1)
  
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

def plot_multiclass_roc(clf,x_test, y_test, n_classes, figsize=(17, 10)):

    # structures
    fpr = dict()
    tpr = dict()
    
    roc_auc = dict()
    y_score1 = clf.predict(x_test)
    # calculate dummies once
    for i in range(n_classes):
        fpr[i], tpr[i], _ = sklearn.metrics.roc_curve(y_test[:, i].ravel(), y_score1[:, i])
        roc_auc[i] = sklearn.metrics.auc(fpr[i], tpr[i])

    # roc for each class
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic example')
    for i in range(n_classes):
        ax.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f) for label %s' % (roc_auc[i], labels[i]))
    ax.legend(loc="best")
    ax.grid(alpha=.4)
    sns.despine()
    plt.show()
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
def plot_metrics(history):
    import matplotlib.pyplot as plt
    metrics = ['loss', 'auc', 'precision', 'recall']
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        plt.subplot(2,2,n+1)
        plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
        plt.plot(history.epoch, history.history['val_'+metric],
                 color=colors[0], linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        if metric == 'loss':
            plt.ylim([0, plt.ylim()[1]])
        elif metric == 'auc':
             plt.ylim([0.8,1])
        else:
            plt.ylim([0,1])

        plt.legend()

In [ ]:
#per_char=pd.read_csv('/content/drive/My Drive/Char_perm.csv')
per_level=pd.read_csv('/content/drive/My Drive/permissions.csv')
import  tensorflow as tf 
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
encode_label=encoder.fit_transform(per_level['type'])
labels_full=to_categorical(encode_label)
labels=np.unique(per_level.type).tolist()
labels

In [ ]:

per_level['permmisions']=per_level['permmisions'].apply(lambda x:str(x))
permiss=per_level['permmisions']
Count_perm=per_level['permmisions'].apply(lambda x:len(x.strip().replace('android.permission.','').replace('android.','').replace(',','.').replace('_','.').split('.')))
from keras.preprocessing.sequence import pad_sequences
max_length = max([len(s.replace('.','-').split('_')) for s in permiss])
max_length


In [ ]:
from keras.preprocessing.text import Tokenizer
def tokn(text):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(text)

    train = tokenizer.texts_to_sequences(text)


    vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
    vocab_size
    return train,vocab_size
train,vocab_size_word=tokn(permiss)
print (vocab_size_word)
perm_train,perm_test, y_train, y_test = train_test_split( train, labels_full, test_size=0.33,random_state=42,shuffle=True)

In [ ]:

encoded_docs,vocab_size_word=tokn(permiss)
perm_train,perm_test, y_train, y_test = train_test_split( encoded_docs, labels_full, test_size=0.33,shuffle=True)
from keras.preprocessing.sequence import pad_sequences
maxlen = 249

X_train = pad_sequences(perm_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(perm_test, padding='post', maxlen=maxlen)

def conv_1d():
    model_c1 = Sequential()
    model_c1.add(Embedding(vocab_size_word, 110, input_length=maxlen))
    
    model_c1.add(Dropout(0.4))
   
    model_c1.add(Convolution1D(filters=128, kernel_size=3,padding='same', activation='relu'))
    model_c1.add(MaxPooling1D(pool_size=3))
    model_c1.add(Dropout(0.5))
    model_c1.add(Convolution1D(filters=128, kernel_size=3,padding='same', activation='relu'))
    model_c1.add(MaxPooling1D(pool_size=3))
    model_c1.add(Dropout(0.5))
    model_c1.add(Flatten())
    
    model_c1.add(Dense(100, activation='relu'))

    model_c1.add(Dense(5, activation='softmax'))
    metrics1=[tf.keras.metrics.TruePositives(name='tp'),
              tf. keras.metrics.FalsePositives(name='fp'),
              tf.keras.metrics.TrueNegatives(name='tn'),
              tf.keras.metrics.FalseNegatives(name='fn'),
              tf. keras.metrics.CategoricalAccuracy(name='accuracy'),
              tf.keras.metrics.Precision(name='precision'),
              tf.keras.metrics.Recall(name='recall'),
              tf.keras.metrics.AUC(name='auc')]

    model_c1.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(0.001,beta_1=0.9, beta_2=0.9, epsilon=1e-03), metrics=metrics1)
    return model_c1




model_c1=conv_1d()
metrics2=["accuracy"]
start = time.process_time()
# fit network
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001)
#model.fit(X_train, Y_train, callbacks=[reduce_lr])
history_c1=model_c1.fit(X_train,y_train,verbose=1,batch_size=256,epochs=50,validation_data=(X_test,y_test),callbacks=[reduce_lr])

In [ ]:
!pip install pretty_confusion_matrix

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
Y_prediction = model_c1.predict(X_test, verbose=1)
# Convert predictions classes to one hot vectors 


Y_pred_classes = np.argmax(Y_prediction,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)

df_cm = pd.DataFrame(confusion_mtx, index=np.unique(per_level.type.values), columns=np.unique(per_level.type.values))
from pretty_confusion_matrix import pp_matrix
cmap = 'PuRd'
cm=pp_matrix(df_cm,cmap=plt.cm.CMRmap_r)


In [ ]:
y=np.argmax(Y_pred_classes,axis=1)

print(f1_score(Y_true,y,average='micro'))

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier
maxlen = 249

X_train = pad_sequences(perm_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(perm_test, padding='post', maxlen=maxlen)

#maxlen_char  = max_length_char
#X_train_char = pad_sequences(perm_train_ch, padding='post', maxlen=maxlen_char)
#X_test_char = pad_sequences(perm_test_ch, padding='post', maxlen=maxlen_char)

def model_lstm(embedding_dim ,vocab_size,maxlen):
      model = Sequential()
      model.add(Embedding(input_dim=vocab_size, 
                                output_dim=embedding_dim, 
                                input_length=maxlen))
      model.add(SpatialDropout1D(0.3))

      model.add((LSTM(128,dropout=0.5,return_sequences=True)))

      model.add(SpatialDropout1D(0.6))


      model.add((LSTM(128,dropout=0.5, return_sequences=True)))
      model.add(SpatialDropout1D(0.7))
      
      


      model.add(Flatten())
      #model.add(RandomFourierFeatures(output_dim=500, scale=1.0, kernel_initializer="gaussian"))
      model.add(Dense(maxlen, activation='relu'))
      model.add(Dropout(0.8))
      model.add(Dense(5, activation='softmax'))
      return model
epochs = 50
model=model_lstm(110,vocab_size_word,maxlen)
batch_size = 256
metrics1=[tf.keras.metrics.TruePositives(name='tp'),
            tf. keras.metrics.FalsePositives(name='fp'),
            tf.keras.metrics.TrueNegatives(name='tn'),
            tf.keras.metrics.FalseNegatives(name='fn'),
            tf. keras.metrics.CategoricalAccuracy(name='accuracy'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.AUC(name='auc')]
metrics2=["accuracy"]
model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(0.001,beta_1=0.9, beta_2=0.9, epsilon=1e-03), metrics= metrics1)
model.summary()
#print(model.layers[0].get_weights()[0])
import time
start = time.process_time()
print(start)
history_lstm = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_data=(X_test, y_test),callbacks=[EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)])

#loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
#print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history_lstm)
end = time.process_time()
print("Time for LSTM model : {} ".format((end-start)))


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
Y_prediction = model.predict(X_test, verbose=1)
# Convert predictions classes to one hot vectors 


Y_pred_classes = np.argmax(Y_prediction,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)

df_cm = pd.DataFrame(confusion_mtx, index=np.unique(per_level.type.values), columns=np.unique(per_level.type.values))
from pretty_confusion_matrix import pp_matrix
cmap = 'PuRd'
cm=pp_matrix(df_cm,cmap=plt.cm.CMRmap_r)


In [ ]:

y=np.argmax(Y_pred_classes,axis=1)

print(f1_score(Y_true,y,average='micro'))

In [ ]:
end = time.process_time()
print("Time for LSTM model : {} ".format((end-start)))

In [ ]:
from keras.preprocessing.sequence import pad_sequences
maxlen = 249
epochs = 50
batch_size = 256
X_train = pad_sequences(perm_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(perm_test, padding='post', maxlen=maxlen)

#maxlen_char  = max_length_char
#X_train_char = pad_sequences(perm_train_ch, padding='post', maxlen=maxlen_char)
#X_test_char = pad_sequences(perm_test_ch, padding='post', maxlen=maxlen_char)

def model_lstm(embedding_dim,vocab_size,maxlen,opt):
      model = Sequential()
      model.add(Embedding(input_dim=vocab_size, 
                                output_dim=embedding_dim, 
                                input_length=maxlen))
      model.add(SpatialDropout1D(0.3))

      model.add((LSTM(128,dropout=0.5,return_sequences=True)))

      model.add(SpatialDropout1D(0.6))


      model.add((LSTM(128,dropout=0.5, return_sequences=True)))
      model.add(SpatialDropout1D(0.7))
      
      


      model.add(Flatten())
      #model.add(RandomFourierFeatures(output_dim=500, scale=1.0, kernel_initializer="gaussian"))
      model.add(Dense(maxlen, activation='relu'))
      model.add(Dropout(0.8))
      model.add(Dense(5, activation='softmax'))
     
      metrics1=[tf.keras.metrics.TruePositives(name='tp'),
            tf. keras.metrics.FalsePositives(name='fp'),
            tf.keras.metrics.TrueNegatives(name='tn'),
            tf.keras.metrics.FalseNegatives(name='fn'),
            tf. keras.metrics.CategoricalAccuracy(name='accuracy'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.AUC(name='auc')]
      metrics2=["accuracy"]
      model.compile(loss='categorical_crossentropy',optimizer=opt, metrics= metrics1)
      return model
for m in ['Adam','RMSprop','SGD','Nadam']:
    for batch in [32,64,1,28,256,512,1024]:
        model =model_lstm(110,vocab_size_word,maxlen,m)

    #print(model.layers[0].get_weights()[0])
        import time
        start = time.process_time()
        print(start)
        history_lstm = model.fit(X_train, y_train, epochs=epochs, batch_size=batch,validation_data=(X_test, y_test),callbacks=[EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)])

        #loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
        #print("Testing Accuracy:  {:.4f}".format(accuracy))
        plot_history_opt(history_lstm,m,str(batch))
        end = time.process_time()
        print("Time for LSTM model : {} ".format((end-start)))


In [ ]:

encoded_docs,vocab_size_word=tokn(permiss)
perm_train,perm_test, y_train, y_test = train_test_split( encoded_docs, labels_full, test_size=0.33,shuffle=True)
from keras.preprocessing.sequence import pad_sequences
maxlen = 249

X_train = pad_sequences(perm_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(perm_test, padding='post', maxlen=maxlen)

def conv_1d(opt):
    model_c1 = Sequential()
    model_c1.add(Embedding(vocab_size_word, 110, input_length=maxlen))
    
    model_c1.add(Dropout(0.4))
   
    model_c1.add(Convolution1D(filters=128, kernel_size=3,padding='same', activation='relu'))
    model_c1.add(MaxPooling1D(pool_size=3))
    model_c1.add(Dropout(0.5))
    model_c1.add(Convolution1D(filters=128, kernel_size=3,padding='same', activation='relu'))
    model_c1.add(MaxPooling1D(pool_size=3))
    model_c1.add(Dropout(0.5))
    model_c1.add(Flatten())
    
    model_c1.add(Dense(100, activation='relu'))

    model_c1.add(Dense(5, activation='softmax'))
    metrics1=[tf.keras.metrics.TruePositives(name='tp'),
              tf. keras.metrics.FalsePositives(name='fp'),
              tf.keras.metrics.TrueNegatives(name='tn'),
              tf.keras.metrics.FalseNegatives(name='fn'),
              tf. keras.metrics.CategoricalAccuracy(name='accuracy'),
              tf.keras.metrics.Precision(name='precision'),
              tf.keras.metrics.Recall(name='recall'),
              tf.keras.metrics.AUC(name='auc')]

    model_c1.compile(loss='categorical_crossentropy', optimizer=opt, metrics=metrics1)
    return model_c1


for opt in ['Adam','RMSprop','SGD','Nadam']:
   for batch in [32,64,128,256,512,1024]:

        model_c1=conv_1d(opt)
        metrics2=["accuracy"]
        start = time.process_time()
        # fit network
        from keras.callbacks import ReduceLROnPlateau
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001)
        #model.fit(X_train, Y_train, callbacks=[reduce_lr])
        history_c1=model_c1.fit(X_train,y_train,verbose=1,batch_size=batch,epochs=50,validation_data=(X_test,y_test),callbacks=[reduce_lr])
        plot_history_opt(history_c1,opt,str(batch))

In [ ]:
per_level['permmisions']=per_level['permmisions'].apply(lambda x:str(x).lower())
permiss=per_level['permmisions']
perm_split=per_level['permmisions'].apply(lambda x:x.replace(',','').replace('android.permission.',' ').strip().split('.')[-1])
lis_value=set()
lis_u=set()
lis_length=[ ]
for perm in perm_split:
    for p in perm.split(' ') :
        if p!='' or p!='nan' :
             lis_u.add(p)

dic={}
for i,perm in enumerate(perm_split):
      lis_value=[ ]
      for p in list(lis_u) :
           if p in perm:
              lis_value.append(1) 
           else:
               lis_value.append(0) 
      dic.update({i:lis_value})
data=pd.DataFrame(dic.values(),columns=list(lis_u))


In [ ]:

X_train, X_test, y_train, y_test = train_test_split( data.values(),labels_full, test_size=0.33,shuffle=True)
X_train=np.expand_dims(X_train,-1)
X_test=np.expand_dims(X_test,-1)
X_train.shape,X_test.shape

In [ ]:

   

def Lst_model_binary(X_t):
    model=Sequential()  
    model.add((LSTM(128,dropout=0.5,input_shape=X_t.shape[1:],return_sequences=True)))
    model.add(SpatialDropout1D(0.3))

    model.add((LSTM(128,dropout=0.5,return_sequences=True)))

    model.add(SpatialDropout1D(0.6))


    model.add((LSTM(128,dropout=0.5, return_sequences=True)))
    model.add(SpatialDropout1D(0.7))
    
    model.add(Flatten())

    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(5, activation='softmax'))
   
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
                            patience=5, verbose=1, mode='auto')
    metrics1=[tf.keras.metrics.TruePositives(name='tp'),
            tf. keras.metrics.FalsePositives(name='fp'),
            tf.keras.metrics.TrueNegatives(name='tn'),
            tf.keras.metrics.FalseNegatives(name='fn'),
            tf. keras.metrics.CategoricalAccuracy(name='accuracy'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.AUC(name='auc')]
    model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(0.001,beta_1=0.9, beta_2=0.9, epsilon=1e-03), metrics= metrics1)
    return model

for batch in [32,64,128,256,512,1024]:
    model =Lst_model_binary(X_train)
    print('optmizer'+ m+str(batch))
#print(model.layers[0].get_weights()[0])
    import time
    start = time.process_time()
    print(start)
    history_lstm = model.fit(X_train, y_train, epochs=50, batch_size=batch,validation_data=(X_test, y_test),callbacks=[EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)])
    end = time.process_time()
    print("Time for LSTM model : {} ".format((end-start)))


In [ ]:
lis_value=()
dic={}
for i,perm in enumerate(perm_split):
      lis_value=[ ]
      for p in list(lis_u) :
           if p in perm:
              lis_value.append(1) 
           else:
               lis_value.append(0) 
      dic.update({i:lis_value})
data=pd.DataFrame(dic.values(),columns=list(lis_u))
Full_data=pd.concat([data,per_level['type']],axis=1)

In [ ]:
import  tensorflow as tf 
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
encode_label=encoder.fit_transform(per_level['type'])
labels_full=to_categorical(encode_label)
labels_full.shape

In [ ]:
pip install scikeras[tensorflow]
import scikeras
from scikeras.wrappers import KerasClassifier

In [ ]:
# Use scikit-learn to grid search the batch size and epochs
import numpy
import tensorflow  
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
import scikeras
from scikeras.wrappers import KerasClassifier
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def model_lstm( optimizer="adam" ):
      embedding_dim=110
      vocab_size=vocab_size_word
      model = Sequential()
      model.add(Embedding(input_dim=vocab_size, 
                                output_dim=embedding_dim, 
                                input_length=maxlen))
      model.add(SpatialDropout1D(0.3))

      model.add((LSTM(128,dropout=0.5,return_sequences=True)))

      model.add(SpatialDropout1D(0.6))


      model.add((LSTM(128,dropout=0.5, return_sequences=True)))
      model.add(SpatialDropout1D(0.7))
      
      
      model.add(Flatten())
      #model.add(RandomFourierFeatures(output_dim=500, scale=1.0, kernel_initializer="gaussian"))
      model.add(Dense(maxlen, activation='relu'))
      model.add(Dropout(0.8))
      model.add(Dense(5, activation='softmax'))
      metrics1=[tf.keras.metrics.TruePositives(name='tp'),
            tf. keras.metrics.FalsePositives(name='fp'),
            tf.keras.metrics.TrueNegatives(name='tn'),
            tf.keras.metrics.FalseNegatives(name='fn'),
            tf. keras.metrics.CategoricalAccuracy(name='accuracy'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.AUC(name='auc')]
      metrics2=["accuracy"]
      model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics= metrics2)
      return model
# fix random seed for reproducibility

start=time.process_time()
# split into input (X) and output (Y) variables
X = Full_train
Y =labels_full
# create model
model_grid = KerasClassifier(build_fn=model_lstm, verbose=0)
# define the grid search parameters
optimizers = ['SGD','Adam']
batches=[64,128,256,512,1024]
epoches=[1]
param_grid = dict(optimizer=optimizers,epochs=epoches,batch_size=batches)
grid = GridSearchCV(estimator=model_grid, param_grid=param_grid,cv=5)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
end = time.process_time()
print("Time for LSTM model : {} ".format((end-start)))

In [ ]:
encoded_docs,vocab_size_word=tokn(permiss)
perm_train,perm_test, y_train, y_test = train_test_split( encoded_docs, labels_full, test_size=0.33,shuffle=True)
from keras.preprocessing.sequence import pad_sequences
maxlen = 249

X_train = pad_sequences(perm_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(perm_test, padding='post', maxlen=maxlen)


In [ ]:



def conv_1d():
    model_c1 = Sequential()
    model_c1.add(Embedding(vocab_size_word, 110, input_length=maxlen))
    
    model_c1.add(Dropout(0.4))
   
    model_c1.add(Convolution1D(filters=128, kernel_size=3,padding='same', activation='relu'))
    model_c1.add(MaxPooling1D(pool_size=3))
    model_c1.add(Dropout(0.5))
    model_c1.add(Convolution1D(filters=128, kernel_size=3,padding='same', activation='relu'))
    model_c1.add(MaxPooling1D(pool_size=3))
    model_c1.add(Dropout(0.5))
    model_c1.add(Flatten())
    
    model_c1.add(Dense(100, activation='relu'))

    model_c1.add(Dense(5, activation='softmax'))

    print(model_c1.summary())
    return model_c1

metrics1=[tf.keras.metrics.TruePositives(name='tp'),
              tf. keras.metrics.FalsePositives(name='fp'),
              tf.keras.metrics.TrueNegatives(name='tn'),
              tf.keras.metrics.FalseNegatives(name='fn'),
              tf. keras.metrics.CategoricalAccuracy(name='accuracy'),
              tf.keras.metrics.Precision(name='precision'),
              tf.keras.metrics.Recall(name='recall'),
              tf.keras.metrics.AUC(name='auc')]
model_c1=conv_1d()
metrics2=["accuracy"]
start = time.process_time()
model_c1.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001,beta_1=0.9, beta_2=0.9, epsilon=1e-03), metrics="accuracy")
# fit network
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001)
#model.fit(X_train, Y_train, callbacks=[reduce_lr])

history_c1=model_c1.fit(X_train,y_train,verbose=1,batch_size=256,epochs=50,validation_data=(X_test,y_test),callbacks=[reduce_lr])

In [ ]:
plot_history(history_c1)
end = time.process_time()
print("Time for Conv1d model : {} ".format((end-start)))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
Y_prediction = model_c1.predict(X_test, verbose=1)
# Convert predictions classes to one hot vectors 


Y_pred_classes = np.argmax(Y_prediction,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
df_cm = pd.DataFrame(confusion_mtx, index=np.unique(per_level.type.values), columns=np.unique(per_level.type.values))
from pretty_confusion_matrix import pp_matrix
cmap = 'PuRd'
pp_matrix(df_cm,cmap=plt.cm.CMRmap_r)

In [ ]:
y=np.argmax(Y_prediction,axis=1)
Y_true
print(f1_score(Y_true,y,average='micro'))

In [ ]:
# fix random seed for reproducibility
import numpy
seed = 7
numpy.random.seed(seed)
# load dataset

# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
#from keras.wrappers.scikit_learn import KerasClassifier
import scikeras
from scikeras.wrappers import KerasClassifier
train,vocab_size_word=tokn(permiss)
Full_train = pad_sequences(train, padding='post', maxlen=maxlen)
# Function to create model, required for KerasClassifier
def conv_1d(optimizer='adam'):
    model_c1 = Sequential()
    model_c1.add(Embedding(vocab_size_word, 110, input_length=maxlen))
    
    model_c1.add(Dropout(0.4))
   
    model_c1.add(Convolution1D(filters=128, kernel_size=3,padding='same', activation='relu'))
    model_c1.add(MaxPooling1D(pool_size=3))
    model_c1.add(Dropout(0.5))
    model_c1.add(Convolution1D(filters=128, kernel_size=3,padding='same', activation='relu'))
    model_c1.add(MaxPooling1D(pool_size=3))
    model_c1.add(Dropout(0.5))
    model_c1.add(Flatten())
    
    model_c1.add(Dense(100, activation='relu'))

    model_c1.add(Dense(5, activation='softmax'))

    print(model_c1.summary())
    model_c1.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics="accuracy")
    return model_c1

# split into input (X) and output (Y) variables
X = Full_train
Y =labels_full
# create model
create_model=conv_1d()
model = KerasClassifier(build_fn=create_model,verbose=0)
# define the grid search parameters
epochs= [50]
batches=[64,128,256,512,1024]
#[ 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(epochs=epochs,batch_size=batches)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

**Binary vectorizations were used to convert the context of permissions into binary 0,1 for all categories in Mal-prem dataset **

In [ ]:

per_level['binary']='Malicious'
per_level

for i,m in enumerate (per_level['type']):
    if per_level['type'][i]=='Benign':
        per_level['binary'][i]='B'
    else: per_level['binary'][i]='M'

In [ ]:
lis_value=()
dic={}
for i,perm in enumerate(perm_split):
      lis_value=[ ]
      for p in list(lis_u) :
           if p in perm:
              lis_value.append(1) 
           else:
               lis_value.append(0) 
      dic.update({i:lis_value})
data=pd.DataFrame(dic.values(),columns=list(lis_u))
Full_data=pd.concat([data,per_level['binary']],axis=1)
   

In [ ]:

lis_value=()
dic={}
for i,perm in enumerate(perm_split):
      lis_value=[ ]
      for p in list(lis_u) :
           if p in perm:
              lis_value.append(1) 
           else:
               lis_value.append(0) 
      dic.update({i:lis_value})
data=pd.DataFrame(dic.values(),columns=list(lis_u))
Full_data=pd.concat([data,per_level['type']],axis=1)
   

X_train, X_test, y_train, y_test = train_test_split(data,labels_full, test_size=0.33,shuffle=True)
X_train=np.expand_dims(X_train,-1)
X_test=np.expand_dims(X_test,-1)
X_train.shape,X_test.shape
def Lst_model(X_t,opt):
    model=Sequential()  

    model.add((LSTM(128,dropout=0.5,input_shape=X_t.shape[1:],return_sequences=True)))

    model.add(SpatialDropout1D(0.6))


    model.add((LSTM(128,dropout=0.5, return_sequences=True)))
    model.add(SpatialDropout1D(0.7))
    
    model.add(Flatten())
    #model.add(RandomFourierFeatures(output_dim=500, scale=1.0, kernel_initializer="gaussian"))
    model.add(Dense(maxlen, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(5, activation='softmax'))
    adam = tf.keras.optimizers.Adam(0.0001)
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
                            patience=5, verbose=1, mode='auto')
    metrics1=[tf.keras.metrics.TruePositives(name='tp'),
            tf. keras.metrics.FalsePositives(name='fp'),
            tf.keras.metrics.TrueNegatives(name='tn'),
            tf.keras.metrics.FalseNegatives(name='fn'),
            tf. keras.metrics.CategoricalAccuracy(name='accuracy'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.AUC(name='auc')]
    model.compile(loss='categorical_crossentropy',optimizer=opt, metrics= metrics1)
    return model
for m in ['Adam','RMSprop','SGD','Nadam']:
    for batch in [32,64,1,28,256,512,1024]:
        model =model_lstm(110,vocab_size_word,maxlen,m)

    #print(model.layers[0].get_weights()[0])
        import time
        start = time.process_time()
        print(start)
        history_lstm = model.fit(X_train, y_train, epochs=epochs, batch_size=batch,validation_data=(X_test, y_test),callbacks=[EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)])




In [ ]:
lis_value=()
dic={}
for i,perm in enumerate(perm_split):
      lis_value=[ ]
      for p in list(lis_u) :
           if p in perm:
              lis_value.append(1) 
           else:
               lis_value.append(0) 
      dic.update({i:lis_value})
data=pd.DataFrame(dic.values(),columns=list(lis_u))
Full_data=pd.concat([data,per_level['binary']],axis=1)
   

In [ ]:
'''
per_level['binary']='Malicious'
per_level

for i,m in enumerate (per_level['type']):
    if per_level['type'][i]=='Benign':
        per_level['binary'][i]='B'
    else: per_level['binary'][i]='M'''
    

In [ ]:
#per_char=pd.read_csv('/content/drive/My Drive/Char_perm.csv')
per_level=pd.read_csv('/content/drive/My Drive/permissions.csv')
per_level['permmisions']=per_level['permmisions'].apply(lambda x:str(x).lower())
permiss=per_level['permmisions']
perm_split=per_level['permmisions'].apply(lambda x:x.replace(',','').replace('android.permission.',' ').strip().split('.')[-1])
Full_permissions=[ ]
lis_u=set()
lis_length=[ ]
for perm in perm_split:
    for p in perm.split(' ') :
        if p!='' or p!='nan' :
             lis_u.add(p)
     

In [ ]:

lis_value=()
dic={}
for i,perm in enumerate(perm_split):
      lis_value=[ ]
      for p in list(lis_u) :
           if p in perm:
              lis_value.append(1) 
           else:
               lis_value.append(0) 
      dic.update({i:lis_value})
  
   

In [ ]:
data=pd.DataFrame(dic.values(),columns=list(lis_u))


In [ ]:
'''
per_level['binary']='Malicious'
per_level

for i,m in enumerate (per_level['type']):
    if per_level['type'][i]=='Benign':
        per_level['binary'][i]='B'
    else: per_level['binary'][i]='M'''

In [ ]:
#Full_data=pd.concat([data,per_level['binary']],axis=1)

In [ ]:
#Full_data

In [ ]:


#data=data.drop(['type'],axis=1)

In [ ]:
train_data=data.values


In [ ]:
labels

In [ ]:

X_train, X_test, y_train, y_test = train_test_split( train_data,labels_full, test_size=0.33,shuffle=True)
X_train=np.expand_dims(X_train,-1)
X_test=np.expand_dims(X_test,-1)

In [ ]:



X_train, X_test, y_train, y_test = train_test_split( train_data,labels_full, test_size=0.33,shuffle=True)
X_train=np.expand_dims(X_train,-1)
X_test=np.expand_dims(X_test,-1)
X_train.shape,X_test.shape
def Lst_model(X_t):
    model=Sequential()  

    model.add((LSTM(128,dropout=0.5,input_shape=X_t.shape[1:],return_sequences=True)))

    model.add(SpatialDropout1D(0.6))


    model.add((LSTM(128,dropout=0.5, return_sequences=True)))
    model.add(SpatialDropout1D(0.7))
    
    


    model.add(Flatten())
    #model.add(RandomFourierFeatures(output_dim=500, scale=1.0, kernel_initializer="gaussian"))
    model.add(Dense(maxlen, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(5, activation='softmax'))
    adam = tf.keras.optimizers.Adam(0.0001)
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
                            patience=5, verbose=1, mode='auto')
    metrics1=[tf.keras.metrics.TruePositives(name='tp'),
            tf. keras.metrics.FalsePositives(name='fp'),
            tf.keras.metrics.TrueNegatives(name='tn'),
            tf.keras.metrics.FalseNegatives(name='fn'),
            tf. keras.metrics.CategoricalAccuracy(name='accuracy'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.AUC(name='auc')]
    model.compile( optimizer=adam,loss='categorical_crossentropy',metrics=metrics1)                        
    model.summary()
    return model


In [ ]:
model=Lst_model(X_train)
history_lstm=model.fit(X_train, y_train, epochs =40, batch_size =256, validation_data = (X_test,y_test),shuffle=True)
plot_history(history_lstm)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
Y_prediction = model.predict(X_test,verbose=1,batch_size=256)
y=np.argmax(Y_pred_classes,axis=1)

print(f1_score(Y_true,y,average='micro'))

In [ ]:


def conv_1d(X_t):
    model_c1 = Sequential()
    model_c1.add(Convolution1D(filters=128, kernel_size=3,input_shape=X_t.shape[1:],padding='same', activation='relu'))
    model_c1.add(MaxPooling1D(pool_size=3))
    model_c1.add(Dropout(0.5))
    model_c1.add(Convolution1D(filters=128, kernel_size=3,padding='same', activation='relu'))
    model_c1.add(MaxPooling1D(pool_size=3))
    model_c1.add(Dropout(0.5))
    model_c1.add(Flatten())
    
    model_c1.add(Dense(100, activation='relu'))

    model_c1.add(Dense(5, activation='softmax'))

    print(model_c1.summary())
    return model_c1

metrics1=[tf.keras.metrics.TruePositives(name='tp'),
              tf. keras.metrics.FalsePositives(name='fp'),
              tf.keras.metrics.TrueNegatives(name='tn'),
              tf.keras.metrics.FalseNegatives(name='fn'),
              tf. keras.metrics.CategoricalAccuracy(name='accuracy'),
              tf.keras.metrics.Precision(name='precision'),
              tf.keras.metrics.Recall(name='recall'),
              tf.keras.metrics.AUC(name='auc')]
model_c1=conv_1d(X_train)
metrics2=["accuracy"]
model_c1.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001,beta_1=0.9, beta_2=0.9, epsilon=1e-03), metrics=metrics1)
# fit network
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001)
#model.fit(X_train, Y_train, callbacks=[reduce_lr])

history_c1=model_c1.fit(X_train,y_train,verbose=1,batch_size=256,epochs=50,validation_data=(X_test,y_test),callbacks=[reduce_lr])

In [ ]:
plot_history(history_c1)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
Y_prediction = model_c1.predict(X_test,verbose=1,batch_size=256)
# Convert predictions classes to one hot vectors 


Y_pred_classes = np.argmax(Y_prediction,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
y=np.argmax(Y_pred_classes,axis=1)

print(f1_score(Y_true,y,average='micro'))